# Capstone project scraping

In [48]:
import re
import requests
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np

import geocoder

## Function to get entire listing

In [49]:
def get_listings(result):
    
    listings = []
    
    for d in result.findAll(class_ = "placard-details module"):

        listings.append(d)

    return listings

## Scraping first page only

In [59]:
# Loopnet banned my ip address while scrapping, so had to use a proxy IP address

proxies = {
    'http':'http://192.25.162.203'
    }

url = 'http://www.loopnet.com/for-lease/new-york-ny/retail/'

r = requests.get(url, proxies=proxies)

page = r.content
soup = BeautifulSoup(page, 'lxml')

results = (get_listings(soup))

In [60]:
# Displying the results of the scrapping

results

[<div class="placard-details module">\n<h5 class="listing-address">\n<a class="column-09" href="http://www.loopnet.com/Listing/19621797/134-West-37th-Street-New-York-NY/" ng-click="RedirectToProfile('http://www.loopnet.com/Listing/19621797/134-West-37th-Street-New-York-NY/'); $event.stopPropagation();" title="134 West 37th Street">\n<address itemprop="address" itemscope="" itemtype="http://schema.org/PostalAddress">\n<span class="street-address" itemprop="streetAddress" title="134 West 37th Street">134 West 37th Street</span>\n<span itemprop="addressLocality" title="New York, NY">New York</span>,\r\n    <span itemprop="addressState" title="NY, NY">NY</span>\n<span itemprop="postalCode" title="10018">10018</span>\n</address>\n</a>\n<span class="listing-price column-04">$45-$79.20 SF/Year</span>\n</h5>\n<ul>\n<li>\n<span>Office Bldg</span>\n</li>\n<li>\n<i>2,500 - 10,000</i>\n<span>SF Available</span>\n</li>\n<li>\n<i>2</i>\n<span>Spaces</span>\n</li>\n<li>\n<i>120,000</i>\n<span>SF Bldg

Every listing should include a list of :

1. listing webpage
2. street address, locality, state, postalcode
3. listing price
4. listing type (office building, restaurant, etc)
5. SF available
6. No. of spaces
7. SF building
8. property description

In [186]:
# Functions to get listing details from scrapped website(s)

def get_listing_details(listing_placard):
    
    listing = []
    
    # listing website
    try:
        listing.append(re.findall('href="(.+)" ng-', \
                                  str(listing_placard))[0])
        
         
    except:
        listing.append('')
    
    # listing streetaddress
    try:
        listing.append(re.findall('streetAddress" title="(.+)"', \
                                  str(listing_placard))[0])
    except:
        listing.append('')
     
    # listing address locality
    try:
        listing.append(re.findall('addressLocality" title="(.+)"', \
                                  str(listing_placard))[0])
    except:
        listing.append('')
    
    # listing state
    try:
        listing.append(re.findall('addressState" title="(.+),', \
                                  str(listing_placard))[0])
    except:
        listing.append('')
    
    # listing postcode
    try:
        listing.append(re.findall('postalCode" title="(\d+)"', \
                                  str(listing_placard))[0])
    except:
        listing.append('')
    
    # listing price
    try:
        listing.append(re.findall('listing-price.+>(.+)<', \
                                  str(listing_placard))[0])
    except:
        listing.append('')
    
    # listing type
    try:
        listing.append(re.findall('h5>\n<ul>\n<li>\n<span>(.+)<', \
                                  str(listing_placard))[0])
    except:
        listing.append('')
    
    # listing square feet available
    try:
        listing.append(re.findall('>(.+)</i>\n<span>SF Available', \
                                  str(listing_placard))[0])
    except:
        listing.append('')

    # listing spaces available
    try:
        listing.append(re.findall('>(.+)</i>\n<span>Spaces', \
                                  str(listing_placard))[0])
    except:
        listing.append('')
    
    # listing building square feet
    try:
        listing.append(re.findall('>(.+)</i>\n<span>SF Bldg', \
                                  str(listing_placard))[0])
    except:
        listing.append('')
    
    #listing property description
    try:
        listing.append(re.findall('property-description">((.|\n|\r)*)<', \
                                  str(listing_placard))[0])
    except:
        listing.append('')
    
    return listing

In [62]:
# Appending all the relevant listing details to an element within page_list
# Every element in the list is then a row of a single listing

page_list = []
for i in results:
    page_list.append(get_listing_details(i))

In [63]:
# Nameing the columns
col = ['Web_address', 'Street_address', 'Locality', 'State', 'Postcode', \
       'Price', 'Property_type', 'SF_avail', 'Spaces_avail', 'Bldg_SF', \
       'Prop_desc']

# Converting data into a dataframe
df = pd.DataFrame(page_list, columns= col)
df

,Web_address,Street_address,Locality,State,Postcode,Price,Property_type,SF_avail,Spaces_avail,Bldg_SF,Prop_desc
0,http://www.loopnet.com/Listing/19621797/134-We...,134 West 37th Street,"New York, NY",NY,10018,$45-$79.20 SF/Year,Office Bldg,"2,500 - 10,000",2,"120,000",Extremely desirable office space in the heart ...
1,http://www.loopnet.com/Listing/19852777/517-Co...,517 Columbus Avenue,"New York, NY",NY,10024,$204.60 SF/Year,Restaurant,"2,346",,"2,345",This is the former Jackson Hole space has not ...
2,http://www.loopnet.com/Listing/19584842/76-Woo...,76 Wooster St,"New York, NY",NY,10012,$80-$84 SF/Year,,"3,000 - 5,000",2,"10,000",
3,http://www.loopnet.com/Listing/19632482/117-Ea...,117 East 15th Street,"New York, NY",NY,10003,$110.67-$120 SF/Year,,"3,300 - 6,600",3,"6,600",
4,http://www.loopnet.com/Listing/19872387/250-Hu...,250 Hudson St,"New York, NY",NY,10013,$75-$90 SF/Year,Office Bldg,"3,100 - 15,832",3,"394,424",
5,http://www.loopnet.com/Listing/19872550/485-Ma...,485 Madison Ave,"New York, NY",NY,10022,$54-$550 SF/Year,Office Bldg,"654 - 31,472",3,"302,000",
6,http://www.loopnet.com/Listing/18803035/315-Ma...,315 Madison Ave,"New York, NY",NY,10017,$60-$355 SF/Year,Office Bldg,"1,802 - 3,894",4,"148,370",
7,http://www.loopnet.com/Listing/19872441/315-Hu...,315 Hudson St,"New York, NY",NY,10013,$100-$175 SF/Year,Office Bldg,"3,922 - 5,881",5,"485,577",
8,http://www.loopnet.com/Listing/20002133/261-Fi...,261 Fifth Ave,"New York, NY",NY,10001,Rate,,"1,590 - 21,457",3,"444,716",
9,http://www.loopnet.com/Listing/19271844/100-Br...,100 Broad St,"New York, NY",NY,10004,$94 SF/Year,,"8,000",,"10,000",


In [64]:
# Checking to see if we got all 20 listings on the page
len(page_list)

# End of single page scrap testing

20

## Starting to scrap Loopnet.com for all their NYC listings

In [189]:
# Engineering the web address to account for page numbers
url_template1 = "http://www.loopnet.com/for-lease/ny/retail/"
url_template2 = "/?view=list"

# Maximum available pages on the website
max_pages = 25

# List of possible proxy IP addresses
"""58.176.46.248
4.31.142.200
64.20.45.138"""

# Proxy IP address

proxies = {
    'http':'http://4.31.142.200'
    }

# Starting with page 2 since we have already scrapped first page 
# from the test above

# Added "+1" since python range doesn't include upper limit
for pg in range(2, max_pages+1, 1):
    
    # Engineering the web address for each page
    url = url_template1 + str(pg) + url_template2

    r = requests.get(url, proxies=proxies)
    if str(r) == "<Response [200]>":

        page = r.content
        soup = BeautifulSoup(page, 'lxml')

        results.append(get_listings(soup))
        for i in results:
            page_list.append(get_listing_details(i))

    else:
        print "This url is not responding favorably."
        print "url =", url
        print "response =", r

In [190]:
# Converting data into dataframe

col = ['Web_address', 'Street_address', 'Locality', 'State', \
       'Postcode', 'Price', 'Property_type', 'SF_avail', \
       'Spaces_avail', 'Bldg_SF', 'Prop_desc']

df = pd.DataFrame(page_list, columns= col)

In [191]:
# Checking to see number of listings scraped
df.shape

(1238, 11)

In [192]:
# Saving to csv!
df.to_csv('listings.csv')

In [199]:
# Dropping any duplicates
df1 = df.drop_duplicates()

In [200]:
# Checking to see how many listings remained
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42 entries, 0 to 892
Data columns (total 14 columns):
Web_address            42 non-null object
Street_address         42 non-null object
Locality               42 non-null object
State                  42 non-null object
Postcode               42 non-null object
Price                  33 non-null object
Property_type          42 non-null object
SF_avail               42 non-null object
Spaces_avail           42 non-null object
Bldg_SF                42 non-null object
Prop_desc              42 non-null object
Full_street_address    42 non-null object
Latitude               20 non-null float64
Longitude              20 non-null float64
dtypes: float64(2), object(12)
memory usage: 4.9+ KB


### Comments:

Loopnet.com seemed to have many repeated entries. From an original universe set of 1,238 listings, there were only 42 entries left. This made me suspicious of the quality of data from Loopnet.com. Hence, this dataset was dropped for other sources.

# Scraping from CityFeet. using Selenium and PhantomJS

In [153]:
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import time

In [269]:
# CityFeet.com using AJAX (Asynchronous JavaScript and XML), so Selenium
# with PhantomJS were used to automatically scroll to the bottom

pause = 5

driver = webdriver.PhantomJS(
    executable_path = \
    '/Users/peidacai/Downloads/phantomjs-2.1.1-macosx/bin/phantomjs')

dcap = dict(DesiredCapabilities.PHANTOMJS)

dcap['phantomjs.page.settings.userAgent'] = \
("Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) \
AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.71 Safari/537.36")

driver = webdriver.PhantomJS(
    executable_path='/Users/peidacai/Downloads/\
    phantomjs-2.1.1-macosx/bin/phantomjs', \
    desired_capabilities = dcap, service_args = [
        '--ignore-ssl-errors=true'])

driver.implicitly_wait(20)
driver.set_window_size(1024, 768)
driver.get('http://www.cityfeet.com/cont/new-york-retail-space#')

# Code to automatically wait 5 seconds for new listings to load before 
# scrolling further. Website stopped loading if this is not included

lastHeight = driver.execute_script("return document.body.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(pause)
    newHeight = driver.execute_script("return document.body.scrollHeight")
    if newHeight == lastHeight:
        break
    lastHeight = newHeight

## Appending html messages to list

In [270]:
soup = BeautifulSoup(driver.page_source, "html.parser")

results = soup.find_all('div','property clearfix src-CF level-30')

cityfeet_results =[]
for d in results:
    cityfeet_results.append(d)


In [271]:
len(cityfeet_results)

12

In [272]:
results = soup.find_all('div','property clearfix src-LN level-20')

for d in results:
    cityfeet_results.append(d)

In [273]:
len(cityfeet_results)

1774

In [274]:
results = soup.find_all('div','property clearfix src-LN level-10')

for d in results:
    cityfeet_results.append(d)

In [275]:
len(cityfeet_results)

1774

In [276]:
results = soup.find_all('div','property clearfix src-LN level-40')

for d in results:
    cityfeet_results.append(d)

In [277]:
len(cityfeet_results)

1777

## Defining function to sieve out listing details

In [183]:
def get_cityfeet_details(listing_placard):
    
    listing = []
    
    # listing streetaddress
    try:
        listing.append(re.findall('title="(.+)">', str(listing_placard))[0])
    except:
        listing.append('')
         
    # listing state
    try:
        listing.append((re.findall('<address>\n<div>(.+)</div>', \
                                   str(listing_placard))[0]).strip())
    except:
        listing.append('')
    
    # listing price
    try:
        listing.append(re.findall('"price">(.+)</strong><strong>', \
                                  str(listing_placard))[0])
    except:
        listing.append('')
        
    # listing square feet available
    try:
        listing.append(re.findall('<strong>(.+)</strong>', \
                                  str(listing_placard))[0])
    except:
        listing.append('')
        
    #listing property description
    try:
        listing.append(re.findall(
                '"propertydescription"><p>((.|\n|\r)*)</p>', \
                str(listing_placard))[0])
    except:
        listing.append('')
    
    return listing

In [278]:
# Adding listing details to dataframe

cityfeet_list = []

for i in cityfeet_results:
    cityfeet_list.append(get_cityfeet_details(i))

dfc = pd.DataFrame(cityfeet_list, columns = ['Street_address', 'State', \
                                             'Price', 'SF_avail', \
                                             'Prop_desc'])

In [279]:
dfc.head()

,Street_address,State,Price,SF_avail,Prop_desc
0,327 Guy Lombardo Avenue,"Freeport, NY 11520",Rent Upon Req.,Size Upon Req.,(Busy Intersection on corner of Guy Lombardo a...
1,327 W 57th St. PRIME RESTAURANT-RETAIL FOR LEASE,"New York, NY 10019",$118.75 /sf/year,"3,840 SF",(Amazing Location! Perfect for Restaurant or R...
2,118 West 14th Street Retail,"New York, NY 10011",$140.00 /sf/year,"4,200 SF",(Prime retail at the base of a luxury resident...
3,"Prime Retail, 5th Avenue Ladies Mile","New York, NY 10011",Rent Upon Req.,"2,500 SF",(This impeccably maintained boutique cast iron...
4,BELOW MARKET!! PRIME MEATPACKING 14th ST,"New York, NY 10014",$65.00 /sf/year,"3,100 SF",(Boutique building located in the heart of the...


# Initial data cleaning

## Cleaning "Price" column

In [387]:
# Removing listings without prices
df_c = dfc[dfc['Price']!="Rent Upon Req."]

### Comments:
Lost about 600 entries, from 1,777 to 1,134.

In [389]:
df_c[df_c['Price'].str.contains("/sf/year")].shape

(1134, 5)

### Comments:
All prices are in the same unit, "square feet per year". So we can leave that alone for now.

## Cleaning address

### Zipcodes

In [390]:
# Extracting zipcodes from address
def get_zips(x):
    return re.findall('\d{5}', x)[0]

In [391]:
df_c.loc[:,'Postcode'] = df_c['State'].apply(get_zips)

In [392]:
df_c.head()

,Street_address,State,Price,SF_avail,Prop_desc,Postcode
1,327 W 57th St. PRIME RESTAURANT-RETAIL FOR LEASE,"New York, NY 10019",$118.75 /sf/year,"3,840 SF",(Amazing Location! Perfect for Restaurant or R...,10019
2,118 West 14th Street Retail,"New York, NY 10011",$140.00 /sf/year,"4,200 SF",(Prime retail at the base of a luxury resident...,10011
4,BELOW MARKET!! PRIME MEATPACKING 14th ST,"New York, NY 10014",$65.00 /sf/year,"3,100 SF",(Boutique building located in the heart of the...,10014
10,778 Nostrand Ave,"Brooklyn, NY 11215",$39.54 /sf/year,"1,669 SF",(Prime Retail Space located on the main strip ...,11215
14,5 Court Square,"Long Island City, NY 11101",$35.00 /sf/year,"3,500 SF","(Immediate possession, 2 5 year lease term,...",11101


### Street address

In [393]:
# Function to create a boolean mask to weed out non-conventional addresses
def get_proper_add(x):
    
    # List of 'proper' address endings, to weed out erroneous ones
    proper_add = ['street', 'avenue', 'st', 'st.', 'ave', 'ave.', \
                  'broadway', 'place', 'pl.', 'pl',\
                  'square', 'cresent', 'road', 'highway', 'turnpike', \
                  'boulevard', 'steinway', '9', 'bypass', 'court', \
                  'parkway', 'lane', 'drive', 'americas', 'bowery', 'plaza']
    
    # Changing to all lower cases so it is easier to compare 
    x = x.lower()
    # 
    x = x.split()
    
    # Addresses ending with words in "proper_add" and digits are ok
    if (x[-1] in proper_add) or (re.match('\d+', x[-1])):
        return False
    else:
        return True

In [394]:
# Adding a boolean mask column
df_c['Address_to_correct'] = df_c['Street_address'].apply(get_proper_add)

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [395]:
# Applying the filter to the dataframe
df_d = df_c[df_c['Address_to_correct'] == True]
df_d.shape

(26, 7)

In [396]:
# Inspecting the addresses
df_d

,Street_address,State,Price,SF_avail,Prop_desc,Postcode,Address_to_correct
1,327 W 57th St. PRIME RESTAURANT-RETAIL FOR LEASE,"New York, NY 10019",$118.75 /sf/year,"3,840 SF",(Amazing Location! Perfect for Restaurant or R...,10019,True
2,118 West 14th Street Retail,"New York, NY 10011",$140.00 /sf/year,"4,200 SF",(Prime retail at the base of a luxury resident...,10011,True
26,17 Knolls Crescent,"Bronx, NY 10463",$53.04 /sf/year,905 SF,"(Strip Mall, l)",10463,True
181,4205 Avenue D,"Brooklyn, NY 11203",$39.00 /sf/year,"1,700 - 3,400 SF",(Coldwell Banker Reliable Commercial Division ...,11203,True
375,2913 Avenue W,"Brooklyn, NY 11229",$33.00 /sf/year,"2,200 SF",(This is a brand new development in the vibran...,11229,True
427,1314 Avenue U,"Brooklyn, NY 11229",$49.00 /sf/year,"1,200 SF",(Prime retail strip Avenue U &amp; east 15 str...,11229,True
548,585 Veterans Road W,"Staten Island, NY 10309",$40.00 /sf/year,"1,000 - 2,400 SF",(Strip center with Pep Boys and Dunkin as co-t...,10309,True
593,95 Avenue B,"New York, NY 10009",$89.61 /sf/year,"1,540 SF","(In the east village, ground level, new buildi...",10009,True
657,25 Gardenville Parkway W,"Buffalo, NY 14224",$8.00 /sf/year,"3,532 SF",(Plaza was recently remodeled. Dollar General...,14224,True
834,1224 Avenue J,"Brooklyn, NY 11230",$90.00 /sf/year,"3,300 SF","(Fully renovated prime retail corner, r)",11230,True


In [397]:
# Correcting individual addresses

df_c.loc[1, 'Street_address'] = '327 W 57th St'
df_c.loc[2, 'Street_address'] = '118 West 14th Street'
df_c.loc[214, 'Street_address'] = '26 W 14th Street'
df_c.loc[1336, 'Street_address'] = '1395 Madison Avenue'
df_c.loc[1765, 'Street_address'] = '328 Canal Street'
df_c.loc[1356, 'Street_address'] = '505 Schutt Road'
df_c.loc[1357, 'Street_address'] = '505 Schutt Road'
df_c.loc[1044, 'Street_address'] = '485 Schutt Road'

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [398]:
df_c.head()

,Street_address,State,Price,SF_avail,Prop_desc,Postcode,Address_to_correct
1,327 W 57th St,"New York, NY 10019",$118.75 /sf/year,"3,840 SF",(Amazing Location! Perfect for Restaurant or R...,10019,True
2,118 West 14th Street,"New York, NY 10011",$140.00 /sf/year,"4,200 SF",(Prime retail at the base of a luxury resident...,10011,True
4,BELOW MARKET!! PRIME MEATPACKING 14th ST,"New York, NY 10014",$65.00 /sf/year,"3,100 SF",(Boutique building located in the heart of the...,10014,False
10,778 Nostrand Ave,"Brooklyn, NY 11215",$39.54 /sf/year,"1,669 SF",(Prime Retail Space located on the main strip ...,11215,False
14,5 Court Square,"Long Island City, NY 11101",$35.00 /sf/year,"3,500 SF","(Immediate possession, 2 5 year lease term,...",11101,False


In [401]:
df_c.shape

(1135, 7)

## Reverting to cleaning price column

In [403]:
# Function to get the digits in the price columns
def get_price(x):
    try:
        return re.findall('\d+\.\d\d', x)[0]
    except:
        return np.nan

In [404]:
# Applying function to the price column
df_c['Price_per_sf_per_year'] = df_c['Price'].apply(get_price)

In [406]:
# Sieving out rows with "NaN" price
df_c[df_c['Price_per_sf_per_year'].isnull()]

,Street_address,State,Price,SF_avail,Prop_desc,Postcode,Address_to_correct,Price_per_sf_per_year
214,26 W 14th Street,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [317]:
df_c.head()

,Street_address,State,Price,SF_avail,Prop_desc,Postcode,Address_to_correct,Price_per_sf_per_year
1,327 W 57th St,"New York, NY 10019",$118.75 /sf/year,"3,840 SF",(Amazing Location! Perfect for Restaurant or R...,10019,True,118.75
2,118 West 14th Street,"New York, NY 10011",$140.00 /sf/year,"4,200 SF",(Prime retail at the base of a luxury resident...,10011,True,140.00
4,BELOW MARKET!! PRIME MEATPACKING 14th ST,"New York, NY 10014",$65.00 /sf/year,"3,100 SF",(Boutique building located in the heart of the...,10014,False,65.00
10,778 Nostrand Ave,"Brooklyn, NY 11215",$39.54 /sf/year,"1,669 SF",(Prime Retail Space located on the main strip ...,11215,False,39.54
14,5 Court Square,"Long Island City, NY 11101",$35.00 /sf/year,"3,500 SF","(Immediate possession, 2 5 year lease term,...",11101,False,35.00


In [407]:
# Dropping NaN row (row 214) and row with ambiguous address (row 4)
df_c.drop(4, axis = 0, inplace = True)
df_c.drop(214, axis = 0, inplace = True)
df_c.head()

,Street_address,State,Price,SF_avail,Prop_desc,Postcode,Address_to_correct,Price_per_sf_per_year
1,327 W 57th St,"New York, NY 10019",$118.75 /sf/year,"3,840 SF",(Amazing Location! Perfect for Restaurant or R...,10019,True,118.75
2,118 West 14th Street,"New York, NY 10011",$140.00 /sf/year,"4,200 SF",(Prime retail at the base of a luxury resident...,10011,True,140.00
10,778 Nostrand Ave,"Brooklyn, NY 11215",$39.54 /sf/year,"1,669 SF",(Prime Retail Space located on the main strip ...,11215,False,39.54
14,5 Court Square,"Long Island City, NY 11101",$35.00 /sf/year,"3,500 SF","(Immediate possession, 2 5 year lease term,...",11101,False,35.00
23,16 W 8th Street,"New York, NY 10011",$149.61 /sf/year,"1,163 SF",(Excellent space steps from Washington Square ...,10011,False,149.61


In [408]:
# Confirming there are no more NaN in price column
df_c[df_c['Price_per_sf_per_year'].isnull()]

,Street_address,State,Price,SF_avail,Prop_desc,Postcode,Address_to_correct,Price_per_sf_per_year


## Extracting city

In [409]:
df_c.loc[:,'City'] = df_c["State"].apply(lambda x: (str(x)).split(',')[0])
df_c.head(20)

,Street_address,State,Price,SF_avail,Prop_desc,Postcode,Address_to_correct,Price_per_sf_per_year,City
1,327 W 57th St,"New York, NY 10019",$118.75 /sf/year,"3,840 SF",(Amazing Location! Perfect for Restaurant or R...,10019,True,118.75,New York
2,118 West 14th Street,"New York, NY 10011",$140.00 /sf/year,"4,200 SF",(Prime retail at the base of a luxury resident...,10011,True,140.00,New York
10,778 Nostrand Ave,"Brooklyn, NY 11215",$39.54 /sf/year,"1,669 SF",(Prime Retail Space located on the main strip ...,11215,False,39.54,Brooklyn
14,5 Court Square,"Long Island City, NY 11101",$35.00 /sf/year,"3,500 SF","(Immediate possession, 2 5 year lease term,...",11101,False,35.00,Long Island City
23,16 W 8th Street,"New York, NY 10011",$149.61 /sf/year,"1,163 SF",(Excellent space steps from Washington Square ...,10011,False,149.61,New York
24,3821 Broadway,"New York, NY 10032",$140.00 - $140.24 /sf/year,830 - 966 SF,(Great corner location in a busy area of Washi...,10032,False,140.00,New York
26,17 Knolls Crescent,"Bronx, NY 10463",$53.04 /sf/year,905 SF,"(Strip Mall, l)",10463,True,53.04,Bronx
27,256 W 109th Street,"New York, NY 10025",$205.26 /sf/year,380 SF,"(one story street retail, l)",10025,False,205.26,New York
28,270 E 10th Street,"New York, NY 10009",$65.16 /sf/year,"1,200 SF","(Ready to operate,Please do not talk to any em...",10009,False,65.16,New York
30,120 Christopher Street,"New York, NY 10014",$129.00 /sf/year,"1,200 SF",(1200 sf basement 400 sf 30 seats Vietnam...,10014,False,129.00,New York


### Defining lists with only New York City zipcodes

In [331]:
manhattan_zip = pd.read_csv('manhattan_zip.txt', sep='\n')

In [333]:
brooklyn_zip = pd.read_csv('brooklyn_zip.txt', sep = '\n')

In [334]:
bronx_zip = pd.read_csv('bronx.txt', sep = '\n')

In [335]:
queens_zip = pd.read_csv('queens.txt', sep = '\n')

In [336]:
statten_zip = pd.read_csv('statten.txt', sep ='\n')

In [364]:
statten_zip = statten_zip.T.values.tolist()[0]

In [366]:
manhattan_zip = manhattan_zip.T.values.tolist()[0]

In [367]:
bronx_zip = bronx_zip.T.values.tolist()[0]
queens_zip = queens_zip.T.values.tolist()[0]
brooklyn_zip = brooklyn_zip.T.values.tolist()[0]

In [380]:
# NYC zipcodes less Statten Island
nyc_less_stat_zip = manhattan_zip + bronx_zip + brooklyn_zip + queens_zip

In [381]:
# Saving to txt file for possible future uses
nyc_less_stat_zip = pd.Series(nyc_less_stat_zip)
nyc_less_stat_zip.to_csv('nyc_less_stat_zip.txt')

In [368]:
# All NYC zipcodes
nyc_zip = manhattan_zip + bronx_zip + brooklyn_zip + queens_zip + statten_zip
nyc_zip

[10001,
 10002,
 10003,
 10004,
 10005,
 10006,
 10007,
 10009,
 10010,
 10011,
 10012,
 10013,
 10014,
 10016,
 10017,
 10018,
 10019,
 10020,
 10021,
 10022,
 10023,
 10024,
 10025,
 10026,
 10027,
 10028,
 10029,
 10030,
 10031,
 10032,
 10033,
 10034,
 10035,
 10036,
 10037,
 10038,
 10039,
 10040,
 10044,
 10048,
 10065,
 10069,
 10075,
 10111,
 10115,
 10128,
 10280,
 10281,
 10282,
 10451,
 10452,
 10453,
 10454,
 10455,
 10456,
 10457,
 10458,
 10459,
 10460,
 10461,
 10462,
 10465,
 10466,
 10467,
 10468,
 10469,
 10470,
 10471,
 10472,
 10473,
 10474,
 10475,
 11201,
 11203,
 11204,
 11205,
 11206,
 11207,
 11208,
 11209,
 11210,
 11211,
 11212,
 11213,
 11214,
 11215,
 11216,
 11217,
 11218,
 11219,
 11220,
 11221,
 11222,
 11223,
 11224,
 11225,
 11226,
 11228,
 11229,
 11230,
 11231,
 11232,
 11233,
 11234,
 11235,
 11236,
 11237,
 11238,
 11239,
 11251,
 11001,
 11004,
 11005,
 11040,
 11096,
 11101,
 11102,
 11103,
 11104,
 11105,
 11106,
 11109,
 11354,
 11355,
 11356,


In [370]:
nyc_zip = pd.Series(nyc_zip)
nyc_zip.to_csv('nyc_zip.txt')

### Removing rows without zip code

In [410]:
df_c[df_c['Postcode'].isnull()]

,Street_address,State,Price,SF_avail,Prop_desc,Postcode,Address_to_correct,Price_per_sf_per_year,City


### Checking number of entries in nyc

In [411]:
df_c['Postcode'] = df_c['Postcode'].astype(int)

In [412]:
df_c[df_c['Postcode'].isin(nyc_zip)].shape

(569, 9)

#### Comments:
569 entries in NYC seemed decent enough for modeling. However, due to the relative inaccessibility of Statten Island by taxi, I am considering droppping entries in Statten Island. Ideally, we would have sufficient listings in Manhattan to reduce variability in comparison, since taxi would feature less strongly as a mode of transport in other boroughs. However... (see below)

#### Number of entries in Manhattan

In [413]:
df_c[df_c['Postcode'].isin(manhattan_zip)].shape

(180, 9)

#### Comments:
There are only 180 entries in Manhattan, possibly less, after more cleaning.

#### Number of entries in NYC less Statten Island

In [414]:
df_c[df_c['Postcode'].isin(nyc_less_stat_zip)].shape

(554, 9)

#### Comments:
Thankfully, there were only a few listings in Statten Island, therefore, we can proceed to drop Statten Island listings.

## Cleaning "SF_avail" column

In [415]:
# Removing characters
df_c.loc[:, 'SF_avail'] = \
df_c['SF_avail'].apply(lambda x : x.replace(' SF', ''))

In [416]:
# Removing commas
df_c.loc[:, 'SF_avail'] = \
df_c['SF_avail'].apply(lambda x : x.replace(',', ''))

df_c.head()

,Street_address,State,Price,SF_avail,Prop_desc,Postcode,Address_to_correct,Price_per_sf_per_year,City
1,327 W 57th St,"New York, NY 10019",$118.75 /sf/year,3840,(Amazing Location! Perfect for Restaurant or R...,10019,True,118.75,New York
2,118 West 14th Street,"New York, NY 10011",$140.00 /sf/year,4200,(Prime retail at the base of a luxury resident...,10011,True,140.00,New York
10,778 Nostrand Ave,"Brooklyn, NY 11215",$39.54 /sf/year,1669,(Prime Retail Space located on the main strip ...,11215,False,39.54,Brooklyn
14,5 Court Square,"Long Island City, NY 11101",$35.00 /sf/year,3500,"(Immediate possession, 2 5 year lease term,...",11101,False,35.00,Long Island City
23,16 W 8th Street,"New York, NY 10011",$149.61 /sf/year,1163,(Excellent space steps from Washington Square ...,10011,False,149.61,New York


In [420]:
# Some entries had a range of values for listing floor size
# These would be multiple units for rent in the same address.
# Based on experience, listing would show the lowest price on the website
# as click-bait
# Therefore, only the largest floor area were extracted

def clean_sf(x):
    if type(x) == type(1):
        return int(x)
    else:
        # Returning only the larger floor area of the range 
        # (see comments above)
        return int(x.split('-')[1])

In [421]:
# Applying change to "SF_avail" column
df_cityfeet.loc[:, 'SF_avail'] = df_cityfeet['SF_avail'].apply(clean_sf)

## Forming a full street address (for geocoding)

In [422]:
df_c.head()

,Street_address,State,Price,SF_avail,Prop_desc,Postcode,Address_to_correct,Price_per_sf_per_year,City
1,327 W 57th St,"New York, NY 10019",$118.75 /sf/year,3840,(Amazing Location! Perfect for Restaurant or R...,10019,True,118.75,New York
2,118 West 14th Street,"New York, NY 10011",$140.00 /sf/year,4200,(Prime retail at the base of a luxury resident...,10011,True,140.00,New York
10,778 Nostrand Ave,"Brooklyn, NY 11215",$39.54 /sf/year,1669,(Prime Retail Space located on the main strip ...,11215,False,39.54,Brooklyn
14,5 Court Square,"Long Island City, NY 11101",$35.00 /sf/year,3500,"(Immediate possession, 2 5 year lease term,...",11101,False,35.00,Long Island City
23,16 W 8th Street,"New York, NY 10011",$149.61 /sf/year,1163,(Excellent space steps from Washington Square ...,10011,False,149.61,New York


In [431]:
# Converting zipcodes to str type for concatenation
df_c['zips'] = df_c['Postcode'].astype(str)

In [432]:
# Forming full street address
df_c['Full_Street_address'] = df_c['Street_address'] + ', ' + df_c['City'] + ', NY ' + df_c['zips']
df_c.head()

,Street_address,State,Price,SF_avail,Prop_desc,Postcode,Address_to_correct,Price_per_sf_per_year,City,Full_Street_address,zips
1,327 W 57th St,"New York, NY 10019",$118.75 /sf/year,3840,(Amazing Location! Perfect for Restaurant or R...,10019,True,118.75,New York,"327 W 57th St, New York, NY 10019",10019
2,118 West 14th Street,"New York, NY 10011",$140.00 /sf/year,4200,(Prime retail at the base of a luxury resident...,10011,True,140.00,New York,"118 West 14th Street, New York, NY 10011",10011
10,778 Nostrand Ave,"Brooklyn, NY 11215",$39.54 /sf/year,1669,(Prime Retail Space located on the main strip ...,11215,False,39.54,Brooklyn,"778 Nostrand Ave, Brooklyn, NY 11215",11215
14,5 Court Square,"Long Island City, NY 11101",$35.00 /sf/year,3500,"(Immediate possession, 2 5 year lease term,...",11101,False,35.00,Long Island City,"5 Court Square, Long Island City, NY 11101",11101
23,16 W 8th Street,"New York, NY 10011",$149.61 /sf/year,1163,(Excellent space steps from Washington Square ...,10011,False,149.61,New York,"16 W 8th Street, New York, NY 10011",10011


In [433]:
# Extracting only relevant columns of the dataframe

df_cityfeet = pd.concat([df_c['Full_Street_address'], \
                         df_c['SF_avail'], \
                         df_c['Price_per_sf_per_year'], \
                         df_c['Prop_desc']], \
                        axis = 1)
df_cityfeet.reset_index(inplace = True, drop = True)
df_cityfeet.head()

,Full_Street_address,SF_avail,Price_per_sf_per_year,Prop_desc
0,"327 W 57th St, New York, NY 10019",3840,118.75,(Amazing Location! Perfect for Restaurant or R...
1,"118 West 14th Street, New York, NY 10011",4200,140.00,(Prime retail at the base of a luxury resident...
2,"778 Nostrand Ave, Brooklyn, NY 11215",1669,39.54,(Prime Retail Space located on the main strip ...
3,"5 Court Square, Long Island City, NY 11101",3500,35.00,"(Immediate possession, 2 5 year lease term,..."
4,"16 W 8th Street, New York, NY 10011",1163,149.61,(Excellent space steps from Washington Square ...


In [434]:
# Checking types
df_cityfeet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1133 entries, 0 to 1132
Data columns (total 4 columns):
Full_Street_address      1133 non-null object
SF_avail                 1133 non-null object
Price_per_sf_per_year    1133 non-null object
Prop_desc                1133 non-null object
dtypes: object(4)
memory usage: 35.5+ KB


In [435]:
# Converting floor area to float type
df_cityfeet.loc[:, 'Price_per_sf_per_year'] = \
df_cityfeet['Price_per_sf_per_year'].astype(float)

### Adding latitude and longitude

In [437]:
# Geocoding address with latitude and longitude

lat = []
lon = []
for i in df_cityfeet['Full_Street_address']:
    
    try:
        latlon = geocoder.google(i)
        lat.append(latlon.latlng[0])
        lon.append(latlon.latlng[1])
    
    except:
        lat.append(np.nan)
        lon.append(np.nan)
    
df_cityfeet['Latitude'] = lat
df_cityfeet['Longitude'] = lon
df_cityfeet.head()   

,Full_Street_address,SF_avail,Price_per_sf_per_year,Prop_desc,Latitude,Longitude
0,"327 W 57th St, New York, NY 10019",3840,118.75,(Amazing Location! Perfect for Restaurant or R...,40.767498,-73.984111
1,"118 West 14th Street, New York, NY 10011",4200,140.00,(Prime retail at the base of a luxury resident...,40.737629,-73.997937
2,"778 Nostrand Ave, Brooklyn, NY 11215",1669,39.54,(Prime Retail Space located on the main strip ...,40.672089,-73.950522
3,"5 Court Square, Long Island City, NY 11101",3500,35.00,"(Immediate possession, 2 5 year lease term,...",40.746630,-73.942833
4,"16 W 8th Street, New York, NY 10011",1163,149.61,(Excellent space steps from Washington Square ...,40.732584,-73.997463


## Adding back zipcodes and city to facilitate EDA

In [448]:
city = df_c['City']

In [453]:
zipp = df_c['zips']
zipp

0       10019
1       10011
2       11215
3       11101
4       10011
5       10032
6       10463
7       10025
8       10009
9       10014
10      10011
11      10003
12      10002
13      10002
14      11520
15      11780
16      11772
17      11716
18      11722
19      11743
20      10547
21      11727
22      11211
23      11216
24      11218
25      11225
26      11226
27      11206
28      11206
29      11226
        ...  
1103    10075
1104    10022
1105    10573
1106    10013
1107    14217
1108    14224
1109    11385
1110    12590
1111    10541
1112    11040
1113    11434
1114    10562
1115    10598
1116    10549
1117    11426
1118    10570
1119    11222
1120    11557
1121    10012
1122    10519
1123    12524
1124    12603
1125    14850
1126    11706
1127    10013
1128    13203
1129    10549
1130    10467
1131    11101
1132    10024
Name: zips, dtype: object

In [454]:
# Adding "City" as a separate column, for possible EDA purposes later
df_cityfeet['City'] = city
df_cityfeet.head()

,Full_Street_address,SF_avail,Price_per_sf_per_year,Prop_desc,Latitude,Longitude,City
0,"327 W 57th St, New York, NY 10019",3840,118.75,(Amazing Location! Perfect for Restaurant or R...,40.767498,-73.984111,New York
1,"118 West 14th Street, New York, NY 10011",4200,140.00,(Prime retail at the base of a luxury resident...,40.737629,-73.997937,New York
2,"778 Nostrand Ave, Brooklyn, NY 11215",1669,39.54,(Prime Retail Space located on the main strip ...,40.672089,-73.950522,Brooklyn
3,"5 Court Square, Long Island City, NY 11101",3500,35.00,"(Immediate possession, 2 5 year lease term,...",40.746630,-73.942833,Long Island City
4,"16 W 8th Street, New York, NY 10011",1163,149.61,(Excellent space steps from Washington Square ...,40.732584,-73.997463,New York


In [455]:
# Adding zipcode column to facilitate EDA
df_cityfeet['Zipcode'] = zipp
df_cityfeet.head()

,Full_Street_address,SF_avail,Price_per_sf_per_year,Prop_desc,Latitude,Longitude,City,Zipcode
0,"327 W 57th St, New York, NY 10019",3840,118.75,(Amazing Location! Perfect for Restaurant or R...,40.767498,-73.984111,New York,10019
1,"118 West 14th Street, New York, NY 10011",4200,140.00,(Prime retail at the base of a luxury resident...,40.737629,-73.997937,New York,10011
2,"778 Nostrand Ave, Brooklyn, NY 11215",1669,39.54,(Prime Retail Space located on the main strip ...,40.672089,-73.950522,Brooklyn,11215
3,"5 Court Square, Long Island City, NY 11101",3500,35.00,"(Immediate possession, 2 5 year lease term,...",40.746630,-73.942833,Long Island City,11101
4,"16 W 8th Street, New York, NY 10011",1163,149.61,(Excellent space steps from Washington Square ...,40.732584,-73.997463,New York,10011


## Basic EDA

In [257]:
df_cityfeet.describe()

,SF_avail,Price_per_sf_per_year,Latitude,Longitude,Zipcode
count,131.000000,131.000000,131.000000,131.000000,131.000000
mean,8403.114504,134.322824,40.755279,-73.978563,10020.282443
std,48950.738279,98.794998,0.038811,0.024065,21.166110
min,279.000000,18.000000,40.703561,-74.012465,10001.000000
25%,800.000000,64.845000,40.724422,-74.000122,10010.000000
50%,1500.000000,105.000000,40.738547,-73.986773,10014.000000
75%,2694.000000,163.930000,40.785749,-73.953785,10028.500000
max,520000.000000,600.000000,40.862353,-73.925560,10128.000000


In [258]:
# Checking for outliers (large commercial projects)
df_cityfeet[df_cityfeet['SF_avail'] > 50000]

,Street_address,SF_avail,Price_per_sf_per_year,Prop_desc,Latitude,Longitude,Zipcode
64,"2340 5th Avenue, NY 10037",520000,49.0,(Build to Suit Commercial Development in Harle...,40.816982,-73.934953,10037
67,"2005 2023 Third Avenue, NY 10029",121500,50.0,(Build to Suit Commercial Development in East ...,40.794119,-73.942374,10029
68,"315 E 94th Street, NY 10128",185000,60.0,(Build to Suit Commercial Development in the U...,40.782479,-73.946914,10128


In [458]:
# Checking to see eventual number of listings
df_cityfeet.shape

(1133, 8)

In [459]:
# Dropping duplicates
df_cityfeet = df_cityfeet.drop_duplicates()
df_cityfeet.shape

(1128, 8)

In [460]:
# Saving to csv
df_cityfeet.to_csv('cityfeet_retail_rental.csv')

In [461]:
pd.pivot_table(df_cityfeet, values = ['SF_avail', 'Price_per_sf_per_year'], \
               index = ['Zipcode'], aggfunc='count')

,Price_per_sf_per_year,SF_avail
Zipcode,,
10001,3,3
10002,21,21
10003,11,11
10004,1,1
10005,1,1
10007,3,3
10009,4,4
10010,1,1
10011,14,14


In [267]:
pd.pivot_table(df_cityfeet, values = ['SF_avail', 'Price_per_sf_per_year'], \
               index = ['Zipcode'])

,Price_per_sf_per_year,SF_avail
Zipcode,,
10001,165.000000,2700.000000
10002,104.502667,1413.333333
10003,144.753000,1600.000000
10004,43.200000,1250.000000
10005,25.000000,2750.000000
10007,142.833333,1587.666667
10009,74.805000,1434.000000
10011,133.121250,3304.750000
10012,259.555455,2113.636364


## Resources and references

1. Sources for property listings 
- http://www.loopnet.com/
- http://www.cityfeet.com/